In [1]:
import vertexai
from vertexai.generative_models import GenerativeModel
from google.cloud.aiplatform.private_preview.vertex_agents_v2 import agents, sessions
import json
from vertexai.preview.vision_models import ImageGenerationModel
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image
from reportlab.lib.styles import getSampleStyleSheet
from io import BytesIO
from PIL import Image as PILImage
from IPython.display import display, HTML, Image
import time

In [2]:
# Project setup (ensure your project details are correct)
PROJECT_ID = "your-project-id"  # Replace with your project ID
LOCATION = "us-central1"  # Replace with your desired location
vertexai.init(project=PROJECT_ID, location=LOCATION)

In [3]:
MODEL = "gemini-1.5-flash-001"  # Replace if you're using a different model
IMAGE_MODEL = "imagen-3.0-generate-001" # Image generation model


In [4]:
# Story Generator 
story_generator = GenerativeModel(MODEL, system_instruction=["You are a concise and helpful story writer."])

In [ ]:
# Slide Generator Agent
slide_generator_app = agents.create(
    display_name="Slide Generator",
    instruction="""You create clear, concise slide decks.  Each slide has a title, a short paragraph (max 3 sentences), and bullet points. Separate slides with '=== slide ==='."""
)

In [ ]:
# Slide Refiner Agent (New - for iterative improvement)
slide_refiner_app = agents.create(
    display_name="Slide Refiner",
    instruction="""You review and improve slide decks.  Suggest specific changes for clarity, conciseness, and engagement, focusing on title, description, and takeaways.  Return the full revised slide deck text, with changes clearly marked (e.g., using [addition], [removal], [change: old text -> new text])."""
)

In [ ]:
# Image Description Generator Agent
image_description_app = agents.create(
    display_name="Image Descriptions",
    instruction="""You generate evocative image descriptions for slides, capturing the essence of the content and era."""
)


In [ ]:
# JSON Converter Agent
json_converter_app = agents.create(
    display_name="JSON Converter",
    instruction="""Convert slide text to JSON: ```json
[
  {
    "title": "<title>",
    "description": "<description>",
    "takeaways": ["<takeaway 1>", "<takeaway 2>", "<takeaway 3>"]
  }, ...
]
```"""  
)

In [9]:
def generate_images(slide_data, image_description_session, imagen_model,query):
    for i, d in enumerate(slide_data):
        while True:
            try:
                prompt = image_description_session.create_run(agent=image_description_app,
                    content=f"Topic: {query}.\n Slide title: {d['title']} \n Slide main message: {d['description']} \n Slide bullet points: {d['takeaways']}"
                ).steps[-1].content.parts[0].text

                response = imagen_model.generate_images(
                    prompt=prompt, aspect_ratio="1:1"
                )
                slide_data[i]['image'] = response.images[0]._as_base64_string()
                time.sleep(3)  # Avoid rate limiting
                break
            except Exception as e:
                print(f"Image generation error: {e}")
                time.sleep(3)
    return slide_data

In [10]:
query = "transformer architecture deep learning"  #  story idea


In [11]:
story = story_generator.generate_content(query, generation_config={"max_output_tokens": 8192}).text

In [ ]:
# 2. Generate Initial Slides
slide_generator_session = sessions.create()
slides = slide_generator_session.create_run(agent=slide_generator_app,content=f"Write a slide deck for:\n{story}").steps[-1].content.parts[0].text

In [ ]:
print(slides)

In [ ]:
# 3. Refine Slides 
slide_refiner_session = sessions.create()
for _ in range(2):  
    refinement_suggestions = slide_refiner_session.create_run(agent=slide_refiner_app,content=slides).steps[-1].content.parts[0].text
    print(f"Refinement Suggestions:\n{refinement_suggestions}\n")
    slides = refinement_suggestions  

In [ ]:
print(slides)

In [ ]:
# 4. Convert to JSON
json_converter_session = sessions.create()
slides_json = json_converter_session.create_run(agent=json_converter_app,content=slides).steps[-1].content.parts[0].text

In [16]:
try:
    cleaned_json_string = slides_json.replace("```json", "").replace("```", "").strip()
    slide_data = json.loads(cleaned_json_string)  # Convert to Python list of dicts

except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")
    print(f"Problematic JSON string: {cleaned_json_string}")
    slide_data = []  

In [17]:
slide_data = json.loads(cleaned_json_string)

In [ ]:
print(slide_data)

In [ ]:
# 5. Generate Image Descriptions & Images
if slide_data: 
    imagen_model = ImageGenerationModel.from_pretrained(IMAGE_MODEL)
    image_description_session = sessions.create()
    slide_data_with_images = generate_images(slide_data, image_description_session, imagen_model,query)
else:
    print("Could not generate slides and images due to the invalid JSON.")


In [58]:
slide_data = slide_data_with_images

In [59]:
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image
from reportlab.lib.styles import getSampleStyleSheet
from io import BytesIO

In [60]:
# Create a PDF document
pdf_file = "slides.pdf"
doc = SimpleDocTemplate(pdf_file, pagesize=letter)

In [61]:
story = []
styles = getSampleStyleSheet()

In [ ]:
import base64
for slide_data in slide_data:
    # Add title
    story.append(Paragraph(slide_data['title'], styles['h1']))
    story.append(Spacer(1, 12)) 

    # Handle GeneratedImage
    image_data = BytesIO(base64.b64decode(slide_data['image']))
    img = Image(image_data, width=400, height=300)
    story.append(img)
    story.append(Spacer(1, 12))

    # Add description
    story.append(Paragraph(slide_data['description'], styles['Normal']))
    story.append(Spacer(1, 12))

    # Add takeaways as bullet points
    for takeaway in slide_data['takeaways']:
        story.append(Paragraph(f"- {takeaway}", styles['Normal']))
    story.append(Spacer(1, 24))  

# Build the PDF
doc.build(story)

print(f"PDF created: {pdf_file}")